In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder,  StandardScaler
label = LabelEncoder()
scale = StandardScaler()

from tqdm.notebook import tqdm

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
 
    return df

In [ ]:
train= pd.read_csv('../input/tabular-playground-series-feb-2022/train.csv')
train=reduce_mem_usage(train)

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
train.info()

In [ ]:
test=pd.read_csv('../input/tabular-playground-series-feb-2022/test.csv')
test=reduce_mem_usage(test)


In [ ]:
test.isnull().sum()

In [ ]:
test.info()

In [ ]:
d = {'Salmonella_enterica': 0, 'Enterococcus_hirae': 1, 'Escherichia_coli': 2, 'Streptococcus_pyogenes': 3,
     'Campylobacter_jejuni': 4, 'Streptococcus_pneumoniae': 5, 'Staphylococcus_aureus': 6,
     'Escherichia_fergusonii': 7, 'Bacteroides_fragilis': 8, 'Klebsiella_pneumoniae': 9}
d_inv = {v: k for k, v in d.items()}

In [ ]:
X = train.drop(['target', 'row_id'], axis=1)
y = train['target']
y = y.map(d)

Scaling the dataset

In [ ]:
def scaling(train):
    '''Scaling the Dataset'''
    train_scale = scale.fit_transform(train)
    train_scale = pd.DataFrame(train_scale, columns=train.columns)
    return train_scale

SCALING X

In [ ]:
X = scaling(X)

SPLITTING Test-train data

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.20, random_state=42)

MODEL TRAINING

In [ ]:
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)

In [ ]:
print(f'--------------------------------\n| Training Accuracy   :- {(model.score(X_train, y_train)*100).round(2)}% |')
print(f'--------------------------------\n| Validation Accuracy :- {(model.score(X_test, y_test)*100).round(2)}% |\n--------------------------------')

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-feb-2022/sample_submission.csv')
test = reduce_mem_usage(test)
test.drop('row_id', axis=1, inplace=True)

In [ ]:
test = scaling(test)

In [ ]:
y_pred = model.predict(test)

submission['target'] = y_pred
submission['target'] = submission['target'].map(d_inv)
submission.to_csv('submission.csv', index=False)

In [ ]:
submission

In [ ]:
#Code Reference
#https://www.kaggle.com/kartik2khandelwal/knn-classifier-improved-score